<a href="https://colab.research.google.com/github/julihocc/ulsaPye/blob/master/MACD_102.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Actividad 102**

Grupo: 102-A

**Instrucciones**

1. Consulta Eiselt, H. A., Sandblom, C. (2010). Operations Research: A Model-Based Approach. Germany: Springer Berlin Heidelberg, sección 5.3.
2. Replantea el problema 3 del libro Eiselt & Samblont, capítulo 5, como un problema de flujo con costo mínimo.
3. Resuelve el problema utilizando `pywrapgraph.SimpleMinCostFlow()`
de Google OR Tools.
4. Verifica si tu respuesta es equivalente a la que da el libro. En caso de que no sea así, discute las posibles explicaciones.
Transcribe tu solución en DataLore y comparte el enlace al trabajo de tu equipo.

## Problema 3:

Considerndo el diagrama de la red del ejercicio, determine el camino más corto para llegar del nodo $n_s$ a todos los otros nodos, remplantenadolo como un problema de flujo con costo mínimo.

## Solución:

El libro resuelve el problema con el algoritmo de Dijkstra, con el cual obtiene la ruta más corta  con un costo de $13$ unidades para ir desde el nodo fuente al nodo destino es: $n_s \rightarrow n_2 \rightarrow n_1 \rightarrow n_3 \rightarrow n_4 \rightarrow n_t$ 


In [ ]:
!pip install ortools
from ortools.graph import pywrapgraph
import graphviz as gv
import matplotlib.pyplot as plt
import matplotlib.image as img

### Para replantear este problema a un a solución de optimización de mínimo flujo:

Se deben señalar todos los posibles caminos de un paso que existen tomando como referecnia cada uno de los nodos, por ejemplo, desde el nodo fuente (nodo 0) existen tres posibles trayectorias de un paso que serían $n_0 \rightarrow n_1, n_0 \rightarrow n_2, n_0 \rightarrow n_4$, además de agregar el arco circulatorio $(n_t,n_s)$ con capacidad de $5$ y costo $0$. Después se señalan las trayectorias de un paso de los siguientes nodos y guardamos en el objeto `capacites` y `unit_costs` el flujo de cada uno de los arcos de estas posibles trayectorias de un paso.

Para llevar este problema de camino más corto a un problema de mínimo flujo es importante establecer un flujo unitario en el red y de señalar el nodo fuente y el nodo absorbente.

Esto último se consigue con el objeto `supplies` que para el nodo fuente es $1$, para el nodo absorte es $-1$ y para los demas nodos es $0$, estbaleciendo así un flujo unitario en la red que nace en $n_0$ y se consume en $n_t$

In [ ]:
start_nodes = [0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5]
end_nodes   = [1, 2, 4, 3, 5, 1, 3, 4, 4, 5, 3, 5, 0]
capacities  = [6, 2, 10, 3, 9, 3, 7, 9, 2, 6, 1, 3, 1]
unit_costs  = [6, 2, 10, 3, 9, 3, 7, 9, 2, 6, 1, 3, 0]

supplies = [1, 0, 0, 0, 0, -1]

La función objetivo es minimizar el costo del flujo dentro de la red, al ser un flujo unitario la función objetivo va a  minimizar el costo de la trayectoria.

In [ ]:
# Función obejtivo.
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

# Restricciones
for i in range(0, len(start_nodes)):
  min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],capacities[i], unit_costs[i])
  
for i in range(0, len(supplies)):
  min_cost_flow.SetNodeSupply(i, supplies[i])


El Algoritmo encuentra una solución a la función objetivo con las restricciones establecidas

In [ ]:
# Encuentra el costo mínimo del flujo para ir del nodo fuente (n_0) al nodo abosrbente (n_5).
if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
  print('Minimum cost:', min_cost_flow.OptimalCost())
  print('')
  print('  Arc    Flow / Capacity  Cost')
  for i in range(min_cost_flow.NumArcs()):
    cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
    print('%1s -> %1s   %3s  / %3s       %3s' % (
        min_cost_flow.Tail(i),
        min_cost_flow.Head(i),
        min_cost_flow.Flow(i),
        min_cost_flow.Capacity(i),
        cost))
else:
  print('There was an issue with the min cost flow input.')

Minimum cost: 13

  Arc    Flow / Capacity  Cost
0 -> 1     0  /   6         0
0 -> 2     1  /   2         2
0 -> 4     0  /  10         0
1 -> 3     1  /   3         3
1 -> 5     0  /   9         0
2 -> 1     1  /   3         3
2 -> 3     0  /   7         0
2 -> 4     0  /   9         0
3 -> 4     1  /   2         2
3 -> 5     0  /   6         0
4 -> 3     0  /   1         0
4 -> 5     1  /   3         3
5 -> 0     0  /   1         0


## Conclusiones:


La solución del camino más corto que encuntra la herramienta de ORTools es la misma que encuentra como solución el libro desde el algoritmo de Dijkstra.

Se puede revisar esta trayectoria desde la columna de costo ('Cost') en la impresión de la solución. Los nodos que tienen número diferente de cero es la ruta que debe tomar el flujo unitario para recorrer la trayectoria de mínimo costo más y corta que va desde el nodo fuente ($n_0$) hasta el nodo absorbente ($n_t$).


Una observación adicional es que a simple vista es evidente una trayectoria que tiene un costo igual que el mínimo (costo de 13) pero con una trayectoria de menor cantidad de arcos que es la de $n_s \rightarrow n_4 \rightarrow n_t$; nos queda la duda porque en ambos algoritmos no encuentra esta ruta de 3 pasos y si señala una ruta de 5 pasos.